In [2]:
import argparse
import random
import numpy as np
from habitat_baselines.config.default import get_config
import habitat
import os
from habitat import Config, logger
from habitat import Config, Dataset
import matplotlib.pyplot as plt
import transformations as tf

In [3]:
import contextlib
import time
from collections import OrderedDict, defaultdict, deque
from typing import Any, Dict, List, Optional
from habitat_baselines.rl.ddppo.algo.ddp_utils import (
    EXIT,
    REQUEUE,
    add_signal_handlers,
    init_distrib_slurm,
    load_interrupted_state,
    requeue_job,
    save_interrupted_state,
)
from habitat_baselines.common.baseline_registry import baseline_registry
#from habitat_baselines.common.env_utils import construct_envs
from habitat_baselines.common.environments import get_env_class
from habitat_baselines.common.rollout_storage import RolloutStorage
from habitat_baselines.common.tensorboard_utils import TensorboardWriter
from habitat_baselines.common.utils import batch_obs, linear_decay
import numpy as np
import torch
import torch.distributed as distrib
import torch.nn as nn
from gym import spaces
from gym.spaces.dict_space import Dict as SpaceDict
from torch.optim.lr_scheduler import LambdaLR
from habitat_baselines.rl.ddppo.policy.resnet_policy import PointNavResNetPolicy
from habitat_baselines.rl.ddppo.algo.ddppo import DDPPO
from habitat_baselines.common.base_trainer import BaseRLTrainer
from habitat_baselines.common.baseline_registry import baseline_registry

In [4]:
from ppotrainer import PPOTrainer
from ddpotrainer import DDPPOTrainer
from constructenv import construct_envs
from env import MyRLEnv, NavRLEnv1, MyRLEnvNew
from habitat import make_dataset
from constructenv import make_env_fn
from vectorenv import VectorEnv
from utils import FrameSkip, FrameStack, draw_top_down_map

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

# Config init

In [6]:
import fileinput
filename = "/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml"
with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:
    for line in file:
        print(line.replace('\"configs/tasks/pointnav_gibson.yaml\"', '\"/habitat-api/configs/tasks/pointnav_gibson.yaml\"'), end='')
with fileinput.FileInput(filename, inplace=True, backup='.bak') as file:        
    for line in file:    
        print(line.replace('GLOO', 'NCCL'), end='')

In [7]:
W = 256#640
H = 256#360
config_paths="/data/challenge_pointnav2020.local.rgbd.yaml"
config = habitat.get_config(config_paths=config_paths)
config.defrost()
config.SIMULATOR.RGB_SENSOR.HEIGHT = H
config.SIMULATOR.RGB_SENSOR.WIDTH = W
config.SIMULATOR.DEPTH_SENSOR.HEIGHT = H
config.SIMULATOR.DEPTH_SENSOR.WIDTH = W
config.DATASET.DATA_PATH = '/data/v1/{split}/{split}.json.gz'
config.TASK.MEASUREMENTS.append("TOP_DOWN_MAP")
config.TASK.SENSORS = ["HEADING_SENSOR", "COMPASS_SENSOR", "GPS_SENSOR", "POINTGOAL_SENSOR", "POINTGOAL_WITH_GPS_COMPASS_SENSOR"]
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.POINTGOAL_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.GPS_SENSOR.DIMENSIONALITY = 3
config.TASK.GPS_SENSOR.GOAL_FORMAT = "CARTESIAN"
config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
config.TASK.AGENT_POSITION_SENSOR.TYPE = "position_sensor"
config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")
config.SIMULATOR.TURN_ANGLE = 0.5
config.SIMULATOR.TILT_ANGLE = 0.5
config.SIMULATOR.FORWARD_STEP_SIZE = 0.0125
config.ENVIRONMENT.MAX_EPISODE_STEPS = 500*20
config.TASK.TOP_DOWN_MAP.MAX_EPISODE_STEPS = 500*20
config.DATASET.SCENES_DIR = '/data'
config.DATASET.SPLIT = 'train'
config.SIMULATOR.SCENE = '/data/gibson/Aldrich.glb'
#config.SIMULATOR_GPU_ID = 0
config.freeze()

In [8]:
config2 = get_config('/habitat-api/habitat_baselines/config/pointnav/ddppo_pointnav.yaml', [])
ii = 0
for i in config2.TASK_CONFIG.keys():
    config2.TASK_CONFIG[i] = config[i]
    ii+=1
config = config2    

In [9]:
config.defrost()
config.TASK_CONFIG.DATASET.DATA_PATH = '/data/v1/{split}/{split}.json.gz'
config.TASK_CONFIG.DATASET.SCENES_DIR = '/data'
config.TASK_CONFIG.DATASET.SPLIT = 'train'
config.TASK_CONFIG.SIMULATOR.SCENE = '/data/gibson/Aldrich.glb'
config.TASK_CONFIG.TASK.GOAL_SENSOR_UUID = 'pos'#'pointgoal_with_gps_compass'
config.NUM_UPDATES = 50000
config.ENV_NAME = 'MyRLEnvNew'
config.NUM_PROCESSES = 6
config.RL.FRAMESKIP = 20
config.RL.FRAMESTACK = 1
config.freeze()

## Construct one env (example)

In [ ]:
print(config.ENV_NAME)
env_class = get_env_class(config.ENV_NAME)
dataset = make_dataset(config.TASK_CONFIG.DATASET.TYPE, config=config.TASK_CONFIG.DATASET)
one_env = env_class(config=config, dataset=dataset)

In [ ]:
obs = one_env.reset()

In [ ]:
observations, rewards, dones, infos = one_env.step(action=3)
top_down_map = draw_top_down_map(infos, one_env.obs["heading"][0], one_env.obs['rgb'][0].shape[0])
plt.rcParams["figure.figsize"] = [10,10]
plt.subplot(1, 2, 1)
plt.imshow(observations['depth'][:,:,0])
plt.subplot(1, 2, 2)
plt.imshow(top_down_map)
print('Env is done: ',dones)
print('Current coordinates: ',[one_env.trux,one_env.truy])
print('Goal coordinates: ',[one_env.goalx, one_env.goaly])
print('Distance to goal: ',observations['pos'])

## Construct multiple envs with VectorEnv (example)

In [ ]:
num_processes = config.NUM_PROCESSES
configs = [config.clone() for i in range(num_processes)]
env_class = get_env_class(config.ENV_NAME)
env_classes = [env_class for _ in range(num_processes)]

envs = VectorEnv(make_env_fn=make_env_fn,
                env_fn_args=tuple(tuple(zip(configs, env_classes, range(num_processes)))))

In [ ]:
obs = envs.reset()

In [ ]:
outputs = envs.step([2,2,2,2])

observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]

OBSERVATION_SPACE_COMMAND = 'obs'
for write_fn in envs._connection_write_fns:
    write_fn((OBSERVATION_SPACE_COMMAND, None))
obs = [read_fn() for read_fn in envs._connection_read_fns]

OBSERVATION_SPACE_COMMAND = ['trux','truy','goalx','goaly']
for i,command in enumerate(OBSERVATION_SPACE_COMMAND):
    for write_fn in envs._connection_write_fns:
        write_fn((command, None))
    OBSERVATION_SPACE_COMMAND[i] = [read_fn() for read_fn in envs._connection_read_fns]

figure, axes = plt.subplots(nrows=2, ncols=config.NUM_PROCESSES)
for i in range(config.NUM_PROCESSES):
    top_down_map = draw_top_down_map(infos[i], obs[i][0]["heading"][0], obs[i][0]['rgb'][0].shape[0])
    axes[0, i].imshow(observations[i]['depth'][:,:,0])
    axes[1, i].imshow(top_down_map)
    
print('Env is done: ',dones)
print('Current coordinates: ',list(map(list, zip(*OBSERVATION_SPACE_COMMAND[:2]))))
print('Goal coordinates: ',list(map(list, zip(*OBSERVATION_SPACE_COMMAND[2:]))))
print('Distance to goal: ',[list(observations[i]['pos']) for i in range(config.NUM_PROCESSES)])

## How to communicate with VectorEnv manually (example)

In [ ]:
data = [0,0,0,0]
STEP_COMMAND = "step"

if isinstance(data[0], (int, np.integer, str)):
    data = [{"action": {"action": action}} for action in data]
envs._is_waiting = True
for write_fn, args in zip(envs._connection_write_fns, data):
    write_fn((STEP_COMMAND, args))

outputs = []
for read_fn in envs._connection_read_fns:
    outputs.append(read_fn())
observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]    
envs._is_waiting = False
plt.imshow(observations[1]['depth'][:,:,0])

In [ ]:
OBSERVATION_SPACE_COMMAND = 'obs'
for write_fn in trainer.envs._connection_write_fns:
    write_fn((OBSERVATION_SPACE_COMMAND, None))
obs = [read_fn() for read_fn in trainer.envs._connection_read_fns]

In [ ]:
outputs = trainer.envs.step([3 for i in range(config.NUM_PROCESSES)])
observations, rewards, dones, infos = [list(x) for x in zip(*outputs)]

# Run training process

In [10]:
random.seed(config.TASK_CONFIG.SEED)
np.random.seed(config.TASK_CONFIG.SEED)

In [11]:
config.defrost()
config.RL.DDPPO.pretrained_weights = 'gibson-2plus-resnet50.pth'
config.RL.DDPPO.pretrained_encoder = False
config.RL.DDPPO.pretrained = False
config.freeze()

In [12]:
trainer_init = baseline_registry.get_trainer(config.TRAINER_NAME)

In [13]:
trainer = trainer_init(config)

2020-04-12 17:09:16,911 config: BASE_TASK_CONFIG_PATH: /habitat-api/configs/tasks/pointnav_gibson.yaml
CHECKPOINT_FOLDER: data/new_checkpoints
CHECKPOINT_INTERVAL: 50
CMD_TRAILING_OPTS: []
ENV_NAME: MyRLEnvNew
EVAL:
  SPLIT: val
  USE_CKPT_CONFIG: True
EVAL_CKPT_PATH_DIR: data/new_checkpoints
LOG_FILE: train.log
LOG_INTERVAL: 10
NUM_PROCESSES: 6
NUM_UPDATES: 50000
ORBSLAM2:
  ANGLE_TH: 0.2617993877991494
  BETA: 100
  CAMERA_HEIGHT: 1.25
  DEPTH_DENORM: 10.0
  DIST_REACHED_TH: 0.15
  DIST_TO_STOP: 0.05
  D_OBSTACLE_MAX: 4.0
  D_OBSTACLE_MIN: 0.1
  H_OBSTACLE_MAX: 1.25
  H_OBSTACLE_MIN: 0.375
  MAP_CELL_SIZE: 0.1
  MAP_SIZE: 40
  MIN_PTS_IN_OBSTACLE: 320.0
  NEXT_WAYPOINT_TH: 0.5
  NUM_ACTIONS: 3
  PLANNER_MAX_STEPS: 500
  PREPROCESS_MAP: True
  SLAM_SETTINGS_PATH: habitat_baselines/slambased/data/mp3d3_small1k.yaml
  SLAM_VOCAB_PATH: habitat_baselines/slambased/data/ORBvoc.txt
RL:
  DDPPO:
    backbone: resnet50
    distrib_backend: NCCL
    num_recurrent_layers: 2
    pretrained: Fals

In [14]:
trainer.train_init()

2020-04-12 17:09:19,679 Initializing dataset PointNav-v1


0


2020-04-12 17:09:49,472 agent number of trainable parameters: 12454917


In [15]:
pretrained_state = torch.load(config.RL.DDPPO.pretrained_weights, map_location="cpu")
trainer.actor_critic.net.visual_encoder.backbone.load_state_dict({k[len("actor_critic.net.visual_encoder.backbone.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_encoder.backbone" in k})
for param in trainer.actor_critic.net.visual_encoder.backbone.parameters():
    param.requires_grad = False
trainer.actor_critic.net.prev_action_embedding.load_state_dict({k[len("actor_critic.net.prev_action_embedding.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.prev_action_embedding" in k})
for param in trainer.actor_critic.net.prev_action_embedding.parameters():
    param.requires_grad = False
trainer.actor_critic.net.tgt_embeding.load_state_dict({k[len("actor_critic.net.tgt_embeding.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.tgt_embeding" in k})
for param in trainer.actor_critic.net.tgt_embeding.parameters():
    param.requires_grad = False
trainer.actor_critic.net.visual_encoder.compression.load_state_dict({k[len("actor_critic.net.visual_encoder.compression.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_encoder.compression" in k})
for param in trainer.actor_critic.net.visual_encoder.compression.parameters():
    param.requires_grad = False
trainer.actor_critic.net.visual_fc.load_state_dict({k[len("actor_critic.net.visual_fc.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.visual_fc" in k})
for param in trainer.actor_critic.net.visual_fc.parameters():
    param.requires_grad = False
trainer.actor_critic.net.state_encoder.load_state_dict({k[len("actor_critic.net.state_encoder.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.state_encoder" in k})
for param in trainer.actor_critic.net.state_encoder.parameters():
    param.requires_grad = False
trainer.actor_critic.net.state_encoder.load_state_dict({k[len("actor_critic.net.state_encoder.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.net.state_encoder" in k})
for param in trainer.actor_critic.net.state_encoder.parameters():
    param.requires_grad = False
trainer.actor_critic.critic.load_state_dict({k[len("actor_critic.critic.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.critic" in k})
#for param in trainer.actor_critic.critic.parameters():
#    param.requires_grad = False
trainer.actor_critic.action_distribution.load_state_dict({k[len("actor_critic.action_distribution.") :]: v for k, v in pretrained_state["state_dict"].items() if "actor_critic.action_distribution" in k})
#for param in trainer.actor_critic.action_distribution.parameters():
#    param.requires_grad = False

<All keys matched successfully>

In [ ]:
trainer.train()

RESET
{'distance_to_goal': 7.028730773925782, 'spl': 0.0, 'top_down_map.agent_angle': -1.535555076599121}
{'distance_to_goal': 0.0, 'spl': 0.0, 'top_down_map.agent_angle': 0.0}
{'distance_to_goal': 3.6885706583658853, 'spl': 0.0, 'top_down_map.agent_angle': -0.48458099365234375}
{'distance_to_goal': 5.647449970245361, 'spl': 0.0, 'top_down_map.agent_angle': -1.6650047302246094}
{'distance_to_goal': 5.647449970245361, 'spl': 0.0, 'top_down_map.agent_angle': -1.6650047302246094}
{'distance_to_goal': 5.647449970245361, 'spl': 0.0, 'top_down_map.agent_angle': -1.6650047302246094}
{'distance_to_goal': 5.882303025987413, 'spl': 0.0, 'top_down_map.agent_angle': -1.7763037151760526}
{'distance_to_goal': 6.332211529767072, 'spl': 0.0, 'top_down_map.agent_angle': -1.6125813236942999}
{'distance_to_goal': 6.332211529767072, 'spl': 0.0, 'top_down_map.agent_angle': -1.6125813236942999}
{'distance_to_goal': 6.332211529767072, 'spl': 0.0, 'top_down_map.agent_angle': -1.6125813236942999}


2020-04-12 17:22:54,413 update: 10	fps: 10.897	
2020-04-12 17:22:54,416 update: 10	env-time: 731.690s	pth-time: 42.957s	frames: 8448.0
2020-04-12 17:22:54,416 Average window size: 11  distance_to_goal: 6.553  reward: -2.183  spl: 0.000  top_down_map.agent_angle: -1.450


{'distance_to_goal': 6.553165558845766, 'spl': 0.0, 'top_down_map.agent_angle': -1.4504128732988912}
{'distance_to_goal': 6.460648699504573, 'spl': 0.0, 'top_down_map.agent_angle': -1.535490268614234}
{'distance_to_goal': 6.460648699504573, 'spl': 0.0, 'top_down_map.agent_angle': -1.535490268614234}
{'distance_to_goal': 6.460648699504573, 'spl': 0.0, 'top_down_map.agent_angle': -1.535490268614234}
{'distance_to_goal': 6.495611146438954, 'spl': 0.0, 'top_down_map.agent_angle': -1.5976896064226018}
{'distance_to_goal': 6.427370343889509, 'spl': 0.0, 'top_down_map.agent_angle': -1.5581636428833008}
{'distance_to_goal': 6.427370343889509, 'spl': 0.0, 'top_down_map.agent_angle': -1.5581636428833008}
{'distance_to_goal': 6.427370343889509, 'spl': 0.0, 'top_down_map.agent_angle': -1.5581636428833008}
{'distance_to_goal': 6.411245781078673, 'spl': 0.0, 'top_down_map.agent_angle': -1.5604092782003838}
{'distance_to_goal': 6.521736376213305, 'spl': 0.0, 'top_down_map.agent_angle': -1.51265670313

2020-04-12 17:34:51,162 update: 20	fps: 10.810	
2020-04-12 17:34:51,166 update: 20	env-time: 1408.621s	pth-time: 82.290s	frames: 16128.0
2020-04-12 17:34:51,167 Average window size: 21  distance_to_goal: 6.522  reward: -2.265  spl: 0.000  top_down_map.agent_angle: -1.513


{'distance_to_goal': 6.521736376213305, 'spl': 0.0, 'top_down_map.agent_angle': -1.5126567031397964}
{'distance_to_goal': 6.521736376213305, 'spl': 0.0, 'top_down_map.agent_angle': -1.5126567031397964}
{'distance_to_goal': 6.505678546962454, 'spl': 0.0, 'top_down_map.agent_angle': -1.5115600130451259}
{'distance_to_goal': 6.599710836038961, 'spl': 0.0, 'top_down_map.agent_angle': -1.5032792524857954}
{'distance_to_goal': 6.599710836038961, 'spl': 0.0, 'top_down_map.agent_angle': -1.5032792524857954}
{'distance_to_goal': 6.599710836038961, 'spl': 0.0, 'top_down_map.agent_angle': -1.5032792524857954}
{'distance_to_goal': 6.562496909612341, 'spl': 0.0, 'top_down_map.agent_angle': -1.4759353444546084}
{'distance_to_goal': 6.607742823911517, 'spl': 0.0, 'top_down_map.agent_angle': -1.6144956095834797}
{'distance_to_goal': 6.607742823911517, 'spl': 0.0, 'top_down_map.agent_angle': -1.6144956095834797}
{'distance_to_goal': 6.704453178074049, 'spl': 0.0, 'top_down_map.agent_angle': -1.58556332

...

2020-04-13 10:53:24,086 update: 890	fps: 10.725	
2020-04-13 10:53:24,088 update: 890	env-time: 60211.943s	pth-time: 3547.435s	frames: 684288.0
2020-04-13 10:53:24,089 Average window size: 50  distance_to_goal: 6.947  reward: -1.651  spl: 0.006  top_down_map.agent_angle: -1.487


{'distance_to_goal': 6.946763780381945, 'spl': 0.005980855888790554, 'top_down_map.agent_angle': -1.487468465169271}
{'distance_to_goal': 6.930925641741071, 'spl': 0.005915132197704944, 'top_down_map.agent_angle': -1.518508072737809}
{'distance_to_goal': 6.924568684895833, 'spl': 0.005980855888790554, 'top_down_map.agent_angle': -1.5286180284288196}
{'distance_to_goal': 6.906920716002747, 'spl': 0.005915132197704944, 'top_down_map.agent_angle': -1.5053653926639767}
{'distance_to_goal': 6.956325365149456, 'spl': 0.005850837282512499, 'top_down_map.agent_angle': -1.5318991619607676}
{'distance_to_goal': 6.754920054290254, 'spl': 0.006082226327583614, 'top_down_map.agent_angle': -1.5484339827198093}
{'distance_to_goal': 6.759538130326704, 'spl': 0.006116784431717612, 'top_down_map.agent_angle': -1.5415403192693538}
{'distance_to_goal': 6.767444699309593, 'spl': 0.006259035232455232, 'top_down_map.agent_angle': -1.490640684615734}
{'distance_to_goal': 6.775487439385776, 'spl': 0.0061870922

2020-04-13 11:05:24,466 update: 900	fps: 10.724	
2020-04-13 11:05:24,469 update: 900	env-time: 60891.352s	pth-time: 3587.892s	frames: 691968.0
2020-04-13 11:05:24,469 Average window size: 50  distance_to_goal: 6.883  reward: -1.532  spl: 0.006  top_down_map.agent_angle: -1.586


{'distance_to_goal': 6.883380782132768, 'spl': 0.006082226327583614, 'top_down_map.agent_angle': -1.5863450907044492}
{'distance_to_goal': 6.956904970366379, 'spl': 0.006187092298748849, 'top_down_map.agent_angle': -1.5963054086970188}
{'distance_to_goal': 6.936668218568314, 'spl': 0.006259035232455232, 'top_down_map.agent_angle': -1.6369625357694404}
{'distance_to_goal': 7.072515410494942, 'spl': 0.006222855838047976, 'top_down_map.agent_angle': -1.6659806973672326}
{'distance_to_goal': 6.965691226652299, 'spl': 0.006187092298748849, 'top_down_map.agent_angle': -1.6233425798087284}
{'distance_to_goal': 6.9040922062125745, 'spl': 0.0064464314968999984, 'top_down_map.agent_angle': -1.618103758303705}


KeyboardInterrupt: 